In [1]:
import datetime
from pathlib import Path
from netCDF4 import Dataset
from time import strftime,localtime
import numpy as np
createVar = locals()

In [12]:

items=['Solar-Radiation-Flux','Temperature-Air-2m-Min-24h','Temperature-Air-2m-Max-24h', \
       'Vapour-Pressure-Mean', 'Wind-Speed-10m-Mean', 'Precipitation-Flux']
s_items=["rad","tmin","tmax","hum","wind","prec"]
input_path = 'D:/hh/weather'
output_path = 'D:/hh/weather/AgERA5_NC_10_10_grid'
Path(output_path).mkdir(parents=True, exist_ok=True)

parnames = {"rad","tmin","tmax","hum","wind","prec"}
long_name_dict=dict(hum="vapour pressure",rad="irradiation",tmax="maximum temperature",
                    tmin="minimum temperature",wind="mean wind speed",prec="precipitation")
units_dict=dict(hum="kPa",rad="kJ m-2 d-1",tmax="degrees Celsius",
                    tmin="degrees Celsius",wind="m s-1",prec="mm d-1")
dataset_attrs={"Conventions"}
lon_lat_attrs={"standard_name","long_name","units","axis"}
src_file = Path(output_path) / 'temp/hum_073_039.nc'
src_dataset = Dataset(src_file, autoscale=True)

mask_path = Path(input_path) / 'mask_CMFD_V0106_B-01_010deg.nc'
mask_dataset =  Dataset(mask_path, autoscale=True)
mask_data = mask_dataset['mask'][0,:,:][::-1,] #mask数据纬度排序与此相反

for y in range(0,1800,10):
    for x in range(0,3600,10):
        lat,lon = 90-y/10., -180+x/10. #[90,-90],[-180,180],mask数据纬度排序与此相反

        if (y-351<0) or (x-2500<0) or(y-351>=400) or (x-2500>=700):
            continue
        if np.sum(mask_data[y-351:y-341,x-2500:x-2490])==0: #中国大陆相对位置
            continue
        print('.',end='')
        
        data_path = Path(output_path) / ('%03d_%03d.nc'%(lon,lat))
        with Dataset(data_path,"w", format="NETCDF4") as weather_dataset:
            #复制/增加dataset的属性
            for attr in dataset_attrs:
                weather_dataset.__setattr__(attr,src_dataset.__getattribute__(attr))
            weather_dataset.notes="Reorganized by Huang Hai for WOFOST at %s"%strftime("%Y-%m-%d %H:%M:%S", localtime())
            #创建维度
            weather_dataset.createDimension("time",None) 
            weather_dataset.createDimension("lon",10)
            weather_dataset.createDimension("lat",10)
            #创建变量
            times = weather_dataset.createVariable('time', np.float64, ('time',))
            lons = weather_dataset.createVariable('lon', np.float32, ('lon',))
            lats = weather_dataset.createVariable('lat', np.float32, ('lat',))
            for par in s_items:
                createVar[par+"s"]= weather_dataset.createVariable(par, np.float32,('time','lat','lon'))
                                
            #重新设定times的单位
            times.units = "days since 1900-01-01"
            times.calendar = "proleptic_gregorian"
            #赋值经纬度的属性
            for attr in lon_lat_attrs:
                lons.__setattr__(attr,src_dataset.variables["lon"].__getattribute__(attr))
                lats.__setattr__(attr,src_dataset.variables["lat"].__getattribute__(attr))            
 
            for par in s_items:
                createVar["%ss"%par].long_name=long_name_dict[par]
                createVar["%ss"%par].units=units_dict[par]                


            times[:]=np.arange(40177,44194+1,1) #2010-2020
        
            for par in s_items:
                with Dataset(Path(output_path) / ('temp/%s_%03d_%03d.nc'%(par,lon,lat))) as createVar["%s_dataset"%par]:
                    createVar["%s_data"%par]=createVar["%s_dataset"%par].variables[par][:]
                    createVar["%ss"%par][:]=createVar["%s_data"%par]
                    lon_data=createVar["%s_dataset"%par].variables["lon"][:]
                    lat_data=createVar["%s_dataset"%par].variables["lat"][:]
            lons[:]=lon_data
            lats[:]=lat_data                
     
print ('succeed!')               
                

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................